Table 2.   Factors for Insufficient Pain Management, Multivariate Analysis

In [ ]:
# TODO: add winter vs summer and NACA into multivariate analysis 

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/trauma_categories_Rega Pain Study15.09.2025_v2.xlsx'
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/Liste Notärzte-1.xlsx'

In [ ]:
restrict_to_trauma = True

In [ ]:
# Load and preprocess data
data_df = pd.read_excel(data_path)
medic_df = pd.read_excel(medic_data_path)

# Merge with physician data
medic_df = medic_df.drop_duplicates('Mitglieder mit Einsatzfunktion')
medic_df.rename(columns={'Sex m/w': 'physician_sex'}, inplace=True)
data_df = data_df.merge(medic_df, how='left', left_on='Mitglieder mit Einsatzfunktion', right_on='Mitglieder mit Einsatzfunktion')

# Remove duplicates
data_df = data_df.drop_duplicates(subset=["SNZ Ereignis Nr. "])

# Filter patients with VAS > 3 at scene
data_df = data_df[data_df["VAS_on_scene"] > 3]

print(f"Total patients after filtering: {len(data_df)}")
print(f"Adult patients: {(data_df['Alter '] >= 16).sum()}")
print(f"Pediatric patients: {(data_df['Alter '] < 16).sum()}")

In [ ]:
if restrict_to_trauma:
    n_non_trauma = data_df[data_df['Einteilung (reduziert)'] != 'Unfall'].shape[0]
    print(f'Excluded {n_non_trauma} non-trauma patients')

    # adult non-trauma patients
    n_adult_non_trauma = data_df[(data_df['Einteilung (reduziert)'] != 'Unfall') & (data_df["Alter "] >= 16)].shape[0]
    print(f'Excluded {n_adult_non_trauma} adult non-trauma patients')
    # pediatric non-trauma patients
    n_pediatric_non_trauma = data_df[(data_df['Einteilung (reduziert)'] != 'Unfall') & (data_df["Alter "] < 16)].shape[0]
    print(f'Excluded {n_pediatric_non_trauma} pediatric non-trauma patients')

    data_df = data_df[data_df['Einteilung (reduziert)'] == 'Unfall']

In [ ]:
# Prepare variables for logistic regression
df_model = data_df.copy()

# Create outcome variable: insufficient pain management (VAS on arrival > 3)
df_model['insufficient_pain_mgmt'] = (df_model['VAS_on_arrival'] > 3).astype(int)

# Create predictor variables similar to the PDF table
# Age (continuous)
df_model['age'] = df_model['Alter ']

# Sex (male = 1, female = 0)
df_model['male_patient'] = (df_model['Geschlecht'] == 'Männlich').astype(int)

# Physician sex (male = 1, female = 0)
df_model['male_physician'] = (df_model['physician_sex'] == 'm').astype(int)

# Mission type (primary = 1, secondary = 0)
df_model['primary_mission'] = (df_model['Einsatzart'] == 'Primär').astype(int)

# Night mission (night = 1, day = 0)
df_model['night_mission'] = (df_model['Tag oder Nacht'] == 'Nacht').astype(int)

# Trauma (trauma = 1, non-trauma = 0)
df_model['trauma'] = df_model['Einteilung (reduziert)'].str.contains('Unfall', na=False).astype(int)

# Winch extraction (winch = 1, no winch = 0)
df_model['winch_extraction'] = df_model['Bergungen'].str.contains('Winde', na=False).astype(int)

# VAS at scene (continuous)
df_model['vas_scene'] = df_model['VAS_on_scene']

# Mission duration (calculate from timestamps)
df_model['mission_duration'] = (
    pd.to_datetime(df_model['Übergabezeit'], format='%d.%m.%Y %H:%M:%S') - 
    pd.to_datetime(df_model['Erstbefund'], format='%d.%m.%Y %H:%M:%S')
).dt.total_seconds() / 60

# Age groups (pediatric = 1, adult = 0)
df_model['pediatric'] = (df_model['Alter '] < 16).astype(int)

# Short time at scene before transport (dichotomized mission duration)
# Using median split or clinical threshold (e.g., < 30 minutes)
df_model['short_scene_time'] = (df_model['mission_duration'] < 30).astype(int)

# Create medication dose variables (matching Table 1 approach)
df_model['fentanyl_dose'] = 0
df_model['ketamine_dose'] = 0
df_model['esketamine_dose'] = 0
df_model['morphine_dose'] = 0
df_model['Alle Medikamente'] = df_model['Alle Medikamente'].str.replace(',', ';')  # replace commas with semicolons for consistency
for i, row in df_model.iterrows():
    if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
        continue
    for analgetic in row['Alle Medikamente'].split(';'):
        if analgetic.strip() == '':
            continue
        # remove mcg or mg from dose
        if '7IE' in analgetic:
                print(f"Skipping dose with 7IE: {analgetic}")
                continue

        analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
        if 'Fentanyl' in analgetic and '/h' not in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip()
            df_model.at[i, 'fentanyl_dose'] += float(dose) 
        elif 'Fentanyl' in analgetic and '/h' in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
            dose = float(dose) * df_model.at[i, 'mission_duration']  
            df_model.at[i, 'fentanyl_dose'] += float(dose)
        elif 'Ketamin' in analgetic or 'Ketamine' in analgetic:
            dose = analgetic.split('Ketamin')[-1].strip()
            df_model.at[i, 'ketamine_dose'] += float(dose)
        elif 'Esketamin' in analgetic:
            dose = analgetic.split('Esketamin')[-1].strip()
            df_model.at[i, 'esketamine_dose'] += float(dose)
        elif 'Morphin' in analgetic or 'Morphine' in analgetic:
            dose = analgetic.split('Morphin')[-1].strip()
            df_model.at[i, 'morphine_dose'] += float(dose)

# fentanyl given
df_model['fentanyl_given'] = df_model['fentanyl_dose'] > 0
df_model['morphine_given'] = df_model['morphine_dose'] > 0
df_model['ketamine_given'] = df_model['ketamine_dose'] > 0
df_model['esketamine_given'] = df_model['esketamine_dose'] > 0

# Create combined medication variables
df_model['any_opiate_dose'] = df_model['morphine_dose'] + df_model['fentanyl_dose']
df_model['any_opiate_given'] = (df_model['morphine_dose'] > 0) | (df_model['fentanyl_dose'] > 0)
df_model['any_ketamine_given'] = (df_model['ketamine_dose'] > 0) | (df_model['esketamine_dose'] > 0)

# Create no_analgesic variable: patients who receive no analgesic
# (no_analgesic corresponds to a dose of any_opiate and any_ketamine of 0)
df_model['no_analgesic'] = ((df_model['any_opiate_given'] == 0) & (df_model['any_ketamine_given'] == 0)).astype(int)

print(f"Patients receiving no analgesic: {df_model['no_analgesic'].sum()} ({df_model['no_analgesic'].mean():.1%})")
print(f"Patients receiving opiates: {df_model['any_opiate_given'].sum()} ({df_model['any_opiate_given'].mean():.1%})")
print(f"Patients receiving ketamine: {df_model['any_ketamine_given'].sum()} ({df_model['any_ketamine_given'].mean():.1%})")

# Remove rows with missing values in key variables
model_vars = ['insufficient_pain_mgmt', 'age', 'male_patient', 'male_physician', 
              'primary_mission', 'night_mission', 'trauma', 'winch_extraction', 
              'vas_scene', 'mission_duration', 'pediatric', 'short_scene_time', 'no_analgesic']

df_model_clean = df_model[model_vars].dropna()

print(f"Patients included in model: {len(df_model_clean)}")
print(f"Insufficient pain management: {df_model_clean['insufficient_pain_mgmt'].sum()} ({df_model_clean['insufficient_pain_mgmt'].mean():.1%})")
print(f"Sufficient pain management: {(1-df_model_clean['insufficient_pain_mgmt']).sum()} ({(1-df_model_clean['insufficient_pain_mgmt']).mean():.1%})")

In [ ]:
# Univariate logistic regression analysis
def univariate_logistic_regression(df, outcome_var, predictor_vars):
    """
    Perform univariate logistic regression for each predictor variable
    """
    results = []
    
    for var in predictor_vars:
        # Prepare data
        X = df[[var]].copy()
        y = df[outcome_var]
        
        # Add constant for intercept
        X_with_const = sm.add_constant(X)
        
        # Fit logistic regression
        try:
            model = sm.Logit(y, X_with_const).fit(disp=0)
            
            # Extract results
            coef = model.params[var]
            or_value = np.exp(coef)
            ci_lower = np.exp(model.conf_int().loc[var, 0])
            ci_upper = np.exp(model.conf_int().loc[var, 1])
            p_value = model.pvalues[var]
            
            results.append({
                'Variable': var,
                'Coefficient': coef,
                'OR': or_value,
                'CI_lower': ci_lower,
                'CI_upper': ci_upper,
                'P_value': p_value,
                'OR_CI': f"{or_value:.2f} ({ci_lower:.2f}-{ci_upper:.2f})",
                'P_formatted': f"{p_value:.3f}" if p_value >= 0.001 else "<0.001"
            })
            
        except Exception as e:
            print(f"Error with variable {var}: {e}")
            
    return pd.DataFrame(results)

# Define predictor variables for univariate analysis (including the 3 required variables)
predictor_variables = ['age', 'male_patient', 'male_physician', 'short_scene_time',
                      'no_analgesic', 'primary_mission', 'night_mission', 'trauma', 
                      'winch_extraction', 'vas_scene', 'mission_duration', 'pediatric']

# Perform univariate analysis
univariate_results = univariate_logistic_regression(df_model_clean, 'insufficient_pain_mgmt', predictor_variables)

print("Univariate Logistic Regression Results:")
print("=" * 60)
for _, row in univariate_results.iterrows():
    print(f"{row['Variable']}: OR = {row['OR_CI']}, p = {row['P_formatted']}")
    
univariate_results

In [ ]:
# Multivariate logistic regression analysis
def multivariate_logistic_regression(df, outcome_var, predictor_vars):
    """
    Perform multivariate logistic regression
    """
    # Prepare data
    X = df[predictor_vars].copy()
    y = df[outcome_var]
    
    # Add constant for intercept
    X_with_const = sm.add_constant(X)
    
    # Fit logistic regression
    model = sm.Logit(y, X_with_const).fit(disp=0)
    
    # Extract results
    results = []
    for var in predictor_vars:
        coef = model.params[var]
        or_value = np.exp(coef)
        ci_lower = np.exp(model.conf_int().loc[var, 0])
        ci_upper = np.exp(model.conf_int().loc[var, 1])
        p_value = model.pvalues[var]
        
        results.append({
            'Variable': var,
            'Coefficient': coef,
            'OR': or_value,
            'CI_lower': ci_lower,
            'CI_upper': ci_upper,
            'P_value': p_value,
            'OR_CI': f"{or_value:.2f} ({ci_lower:.2f}-{ci_upper:.2f})",
            'P_formatted': f"{p_value:.3f}" if p_value >= 0.001 else "<0.001"
        })
    
    results_df = pd.DataFrame(results)
    
    # Add model statistics
    model_stats = {
        'Log-likelihood': model.llf,
        'AIC': model.aic,
        'BIC': model.bic,
        'Pseudo R-squared': model.prsquared,
        'LLR p-value': model.llr_pvalue
    }
    
    return results_df, model_stats, model

# Select variables for multivariate model - including the 3 required variables
# 1. short_scene_time (short time at scene before transport)
# 2. no_analgesic (patients who receive no analgesic) 
# 3. male_physician (physician sex)
multivariate_vars = ['age', 'male_patient', 'male_physician', 'short_scene_time', 
                    'no_analgesic', 'pediatric', 'primary_mission', 
                    'night_mission', 'trauma', 'vas_scene']

# Perform multivariate analysis
multivariate_results, model_stats, fitted_model = multivariate_logistic_regression(
    df_model_clean, 'insufficient_pain_mgmt', multivariate_vars)

print("Multivariate Logistic Regression Results:")
print("=" * 60)
for _, row in multivariate_results.iterrows():
    print(f"{row['Variable']}: OR = {row['OR_CI']}, p = {row['P_formatted']}")

print(f"\nModel Statistics:")
print(f"Pseudo R-squared: {model_stats['Pseudo R-squared']:.3f}")
print(f"AIC: {model_stats['AIC']:.2f}")
print(f"LLR p-value: {model_stats['LLR p-value']:.3f}")

multivariate_results

In [ ]:
# Create formatted Table 2: Factors for Insufficient Pain Management
def create_table2(univariate_df, multivariate_df):
    """
    Create a publication-ready Table 2
    """
    # Define variable labels for better presentation
    variable_labels = {
        'age': 'Age (years)',
        'male_patient': 'Male patient',
        'male_physician': 'Male physician',
        'short_scene_time': 'Short time at scene (<30 min)',
        'no_analgesic': 'No analgesic given',
        'pediatric': 'Pediatric patient (<16 years)',
        'primary_mission': 'Primary mission',
        'night_mission': 'Night mission',
        'trauma': 'Trauma',
        'winch_extraction': 'Winch extraction',
        'vas_scene': 'VAS at scene',
        'mission_duration': 'Mission duration (minutes)'
    }
    
    # Create combined table
    table2_data = []
    
    # Merge univariate and multivariate results
    for var in univariate_df['Variable']:
        uni_row = univariate_df[univariate_df['Variable'] == var].iloc[0]
        
        # Check if variable is in multivariate model
        multi_row = multivariate_df[multivariate_df['Variable'] == var]
        
        if len(multi_row) > 0:
            multi_row = multi_row.iloc[0]
            multi_or_ci = multi_row['OR_CI']
            multi_p = multi_row['P_formatted']
        else:
            multi_or_ci = '-'
            multi_p = '-'
        
        table2_data.append({
            'Variable': variable_labels.get(var, var),
            'Univariate_OR_CI': uni_row['OR_CI'],
            'Univariate_P': uni_row['P_formatted'],
            'Multivariate_OR_CI': multi_or_ci,
            'Multivariate_P': multi_p
        })
    
    table2_df = pd.DataFrame(table2_data)
    return table2_df

# Create Table 2
table2 = create_table2(univariate_results, multivariate_results)

print("Table 2. Factors for Insufficient Pain Management, Multivariate Analysis")
print("=" * 80)
print(f"{'Variable':<30} {'Univariate':<20} {'P-value':<10} {'Multivariate':<20} {'P-value':<10}")
print(f"{'':30} {'OR (95% CI)':<20} {'':10} {'OR (95% CI)':<20} {'':10}")
print("-" * 80)

for _, row in table2.iterrows():
    print(f"{row['Variable']:<30} {row['Univariate_OR_CI']:<20} {row['Univariate_P']:<10} {row['Multivariate_OR_CI']:<20} {row['Multivariate_P']:<10}")

print("\nOR = Odds Ratio, CI = Confidence Interval")
print(f"Model includes {len(df_model_clean)} patients")
print(f"Insufficient pain management: {df_model_clean['insufficient_pain_mgmt'].sum()}/{len(df_model_clean)} ({df_model_clean['insufficient_pain_mgmt'].mean():.1%})")

# Return the dataframe for further use
table2

In [ ]:
# Model validation and additional statistics
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Get predictions from the multivariate model
y_pred_proba = fitted_model.predict(sm.add_constant(df_model_clean[multivariate_vars]))
y_pred = (y_pred_proba > 0.5).astype(int)
y_true = df_model_clean['insufficient_pain_mgmt']

# Calculate AUC
auc_score = roc_auc_score(y_true, y_pred_proba)

# Classification report
class_report = classification_report(y_true, y_pred, output_dict=True)

print("Model Performance:")
print("=" * 40)
print(f"AUC: {auc_score:.3f}")
print(f"Accuracy: {class_report['accuracy']:.3f}")
print(f"Sensitivity (Recall): {class_report['1']['recall']:.3f}")
print(f"Specificity: {class_report['0']['recall']:.3f}")
print(f"Precision: {class_report['1']['precision']:.3f}")

# Hosmer-Lemeshow test (goodness of fit)
def hosmer_lemeshow_test(y_true, y_prob, n_bins=10):
    """
    Perform Hosmer-Lemeshow goodness-of-fit test
    """
    # Create bins based on predicted probabilities
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]
    
    observed_pos = []
    observed_neg = []
    expected_pos = []
    expected_neg = []
    
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        # Find observations in this bin
        in_bin = (y_prob >= bin_lower) & (y_prob < bin_upper)
        if bin_upper == 1.0:  # Include the maximum value in the last bin
            in_bin = (y_prob >= bin_lower) & (y_prob <= bin_upper)
        
        if in_bin.sum() > 0:
            obs_pos = y_true[in_bin].sum()
            obs_neg = in_bin.sum() - obs_pos
            exp_pos = y_prob[in_bin].sum()
            exp_neg = in_bin.sum() - exp_pos
            
            observed_pos.append(obs_pos)
            observed_neg.append(obs_neg)
            expected_pos.append(exp_pos)
            expected_neg.append(exp_neg)
    
    # Calculate chi-square statistic
    chi_square = 0
    for obs_pos, obs_neg, exp_pos, exp_neg in zip(observed_pos, observed_neg, expected_pos, expected_neg):
        if exp_pos > 0 and exp_neg > 0:
            chi_square += (obs_pos - exp_pos)**2 / exp_pos + (obs_neg - exp_neg)**2 / exp_neg
    
    # Degrees of freedom
    df = len(observed_pos) - 2
    
    # P-value
    p_value = 1 - stats.chi2.cdf(chi_square, df)
    
    return chi_square, df, p_value

# Perform Hosmer-Lemeshow test
hl_chi2, hl_df, hl_p = hosmer_lemeshow_test(y_true, y_pred_proba)

print(f"\nHosmer-Lemeshow Test:")
print(f"Chi-square: {hl_chi2:.3f}")
print(f"Degrees of freedom: {hl_df}")
print(f"P-value: {hl_p:.3f}")
print(f"Interpretation: {'Good fit' if hl_p > 0.05 else 'Poor fit'} (p > 0.05 indicates good fit)")

# Summary statistics for significant variables
print(f"\nSignificant Variables in Multivariate Model (p < 0.05):")
significant_vars = multivariate_results[multivariate_results['P_value'] < 0.05]
if len(significant_vars) > 0:
    for _, row in significant_vars.iterrows():
        direction = "increased" if row['OR'] > 1 else "decreased"
        print(f"- {row['Variable']}: OR = {row['OR']:.2f}, {direction} odds of insufficient pain management")
else:
    print("No variables reached statistical significance (p < 0.05)")

# Save results
# table2.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/table2_results.csv', index=False)
print(f"\nAnalysis complete. Table 2 ready for publication.")

## Separate Analysis: Adult Patients (≥16 years)

In [ ]:
# Prepare adult-only dataset (≥16 years)
adult_df_model = data_df[data_df['Alter '] >= 16].copy()

# Create outcome variable: insufficient pain management (VAS on arrival > 3)
adult_df_model['insufficient_pain_mgmt'] = (adult_df_model['VAS_on_arrival'] > 3).astype(int)

# Create predictor variables for adult patients (exclude pediatric variable)
adult_df_model['age'] = adult_df_model['Alter ']
adult_df_model['male_patient'] = (adult_df_model['Geschlecht'] == 'Männlich').astype(int)
adult_df_model['male_physician'] = (adult_df_model['physician_sex'] == 'm').astype(int)
adult_df_model['primary_mission'] = (adult_df_model['Einsatzart'] == 'Primär').astype(int)
adult_df_model['night_mission'] = (adult_df_model['Tag oder Nacht'] == 'Nacht').astype(int)
adult_df_model['trauma'] = adult_df_model['Einteilung (reduziert)'].str.contains('Unfall', na=False).astype(int)
adult_df_model['winch_extraction'] = adult_df_model['Bergungen'].str.contains('Winde', na=False).astype(int)
adult_df_model['vas_scene'] = adult_df_model['VAS_on_scene']
adult_df_model['mission_duration'] = (
    pd.to_datetime(adult_df_model['Übergabezeit'], format='%d.%m.%Y %H:%M:%S') - 
    pd.to_datetime(adult_df_model['Erstbefund'], format='%d.%m.%Y %H:%M:%S')
).dt.total_seconds() / 60

# Create the same additional variables for adult patients
adult_df_model['short_scene_time'] = (adult_df_model['mission_duration'] < 30).astype(int)

# Create medication dose variables (matching Table 1 approach)
adult_df_model['fentanyl_dose'] = 0
adult_df_model['ketamine_dose'] = 0
adult_df_model['esketamine_dose'] = 0
adult_df_model['morphine_dose'] = 0
adult_df_model['Alle Medikamente'] = adult_df_model['Alle Medikamente'].str.replace(',', ';')  # replace commas with semicolons for consistency
for i, row in adult_df_model.iterrows():
    if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
        continue
    for analgetic in row['Alle Medikamente'].split(';'):
        if analgetic.strip() == '':
            continue
        # remove mcg or mg from dose
        if '7IE' in analgetic:
                print(f"Skipping dose with 7IE: {analgetic}")
                continue

        analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
        if 'Fentanyl' in analgetic and '/h' not in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip()
            adult_df_model.at[i, 'fentanyl_dose'] += float(dose) 
        elif 'Fentanyl' in analgetic and '/h' in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
            dose = float(dose) * adult_df_model.at[i, 'mission_duration']  
            adult_df_model.at[i, 'fentanyl_dose'] += float(dose)
        elif 'Ketamin' in analgetic or 'Ketamine' in analgetic:
            dose = analgetic.split('Ketamin')[-1].strip()
            adult_df_model.at[i, 'ketamine_dose'] += float(dose)
        elif 'Esketamin' in analgetic:
            dose = analgetic.split('Esketamin')[-1].strip()
            adult_df_model.at[i, 'esketamine_dose'] += float(dose)
        elif 'Morphin' in analgetic or 'Morphine' in analgetic:
            dose = analgetic.split('Morphin')[-1].strip()
            adult_df_model.at[i, 'morphine_dose'] += float(dose)

# fentanyl given
adult_df_model['fentanyl_given'] = adult_df_model['fentanyl_dose'] > 0
adult_df_model['morphine_given'] = adult_df_model['morphine_dose'] > 0
adult_df_model['ketamine_given'] = adult_df_model['ketamine_dose'] > 0
adult_df_model['esketamine_given'] = adult_df_model['esketamine_dose'] > 0

# Create combined medication variables
adult_df_model['any_opiate_dose'] = adult_df_model['morphine_dose'] + adult_df_model['fentanyl_dose']
adult_df_model['any_opiate_given'] = (adult_df_model['morphine_dose'] > 0) | (adult_df_model['fentanyl_dose'] > 0)
adult_df_model['any_ketamine_given'] = (adult_df_model['ketamine_dose'] > 0) | (adult_df_model['esketamine_dose'] > 0)

# Create no_analgesic variable: patients who receive no analgesic
# (no_analgesic corresponds to a dose of any_opiate and any_ketamine of 0)
adult_df_model['no_analgesic'] = ((adult_df_model['any_opiate_given'] == 0) & (adult_df_model['any_ketamine_given'] == 0)).astype(int)

# Variables for adult model (including the 3 required variables)
adult_model_vars = ['insufficient_pain_mgmt', 'age', 'male_patient', 'male_physician', 
                   'mission_duration', 'no_analgesic', 'primary_mission', 'night_mission', 
                   'trauma', 'winch_extraction', 'vas_scene']

adult_df_clean = adult_df_model[adult_model_vars].dropna()

print(f"Adult patients included in model: {len(adult_df_clean)}")
print(f"Insufficient pain management: {adult_df_clean['insufficient_pain_mgmt'].sum()} ({adult_df_clean['insufficient_pain_mgmt'].mean():.1%})")
print(f"Sufficient pain management: {(1-adult_df_clean['insufficient_pain_mgmt']).sum()} ({(1-adult_df_clean['insufficient_pain_mgmt']).mean():.1%})")

In [ ]:
# Adult predictor variables (including the 3 required variables)
adult_predictor_vars = ['age', 'male_patient', 'male_physician', 'mission_duration',
                       'no_analgesic', 'primary_mission', 'night_mission', 'trauma', 
                       'winch_extraction', 'vas_scene']

# Univariate analysis for adults
adult_univariate_results = univariate_logistic_regression(adult_df_clean, 'insufficient_pain_mgmt', adult_predictor_vars)

print("ADULT PATIENTS - Univariate Analysis:")
print("=" * 60)
for _, row in adult_univariate_results.iterrows():
    print(f"{row['Variable']}: OR = {row['OR_CI']}, p = {row['P_formatted']}")

# Multivariate analysis for adults (including the 3 required variables)
adult_multivariate_vars = ['age', 'male_patient', 'male_physician', 'mission_duration',
                          'no_analgesic', 'primary_mission', 'night_mission', 'trauma', 'vas_scene', 'winch_extraction']

adult_multivariate_results, adult_model_stats, adult_fitted_model = multivariate_logistic_regression(
    adult_df_clean, 'insufficient_pain_mgmt', adult_multivariate_vars)

print(f"\nADULT PATIENTS - Multivariate Analysis:")
print("=" * 60)
for _, row in adult_multivariate_results.iterrows():
    print(f"{row['Variable']}: OR = {row['OR_CI']}, p = {row['P_formatted']}")

print(f"\nAdult Model Statistics:")
print(f"Pseudo R-squared: {adult_model_stats['Pseudo R-squared']:.3f}")
print(f"AIC: {adult_model_stats['AIC']:.2f}")
print(f"LLR p-value: {adult_model_stats['LLR p-value']:.3f}")

adult_multivariate_results

In [ ]:
# Create Table 2A: Adult Patients
adult_table2 = create_table2(adult_univariate_results, adult_multivariate_results)

print("Table 2A. Factors for Insufficient Pain Management - ADULT PATIENTS (≥16 years)")
print("=" * 80)
print(f"{'Variable':<30} {'Univariate':<20} {'P-value':<10} {'Multivariate':<20} {'P-value':<10}")
print(f"{'':30} {'OR (95% CI)':<20} {'':10} {'OR (95% CI)':<20} {'':10}")
print("-" * 80)

for _, row in adult_table2.iterrows():
    print(f"{row['Variable']:<30} {row['Univariate_OR_CI']:<20} {row['Univariate_P']:<10} {row['Multivariate_OR_CI']:<20} {row['Multivariate_P']:<10}")

print("\nOR = Odds Ratio, CI = Confidence Interval")
print(f"Model includes {len(adult_df_clean)} adult patients")
print(f"Insufficient pain management: {adult_df_clean['insufficient_pain_mgmt'].sum()}/{len(adult_df_clean)} ({adult_df_clean['insufficient_pain_mgmt'].mean():.1%})")

# Show significant variables for adults
adult_significant_vars = adult_multivariate_results[adult_multivariate_results['P_value'] < 0.05]
if len(adult_significant_vars) > 0:
    print(f"\nSignificant Variables in Adult Multivariate Model (p < 0.05):")
    for _, row in adult_significant_vars.iterrows():
        direction = "increased" if row['OR'] > 1 else "decreased"
        print(f"- {row['Variable']}: OR = {row['OR']:.2f}, {direction} odds of insufficient pain management")
else:
    print("\nNo variables reached statistical significance (p < 0.05) in adult model")

adult_table2

## Pediatric Population Analysis (Age < 16 years)

We'll create the same analysis for pediatric patients to understand age-specific factors associated with insufficient pain management.

In [ ]:
# Prepare pediatric dataset (Age < 16 years)
pediatric_df_model = data_df[data_df['Alter '] < 16].copy()

print(f"Pediatric population: {len(pediatric_df_model)} patients")
print(f"Age range: {pediatric_df_model['Alter '].min():.1f} - {pediatric_df_model['Alter '].max():.1f} years")

# Create outcome variable: insufficient pain management (VAS on arrival > 3)
pediatric_df_model['insufficient_pain_mgmt'] = (pediatric_df_model['VAS_on_arrival'] > 3).astype(int)

# Create predictor variables for pediatric patients
pediatric_df_model['age'] = pediatric_df_model['Alter ']
pediatric_df_model['male_patient'] = (pediatric_df_model['Geschlecht'] == 'Männlich').astype(int)
pediatric_df_model['male_physician'] = (pediatric_df_model['physician_sex'] == 'm').astype(int)
pediatric_df_model['primary_mission'] = (pediatric_df_model['Einsatzart'] == 'Primär').astype(int)
pediatric_df_model['night_mission'] = (pediatric_df_model['Tag oder Nacht'] == 'Nacht').astype(int)
pediatric_df_model['trauma'] = pediatric_df_model['Einteilung (reduziert)'].str.contains('Unfall', na=False).astype(int)
pediatric_df_model['winch_extraction'] = pediatric_df_model['Bergungen'].str.contains('Winde', na=False).astype(int)
pediatric_df_model['vas_scene'] = pediatric_df_model['VAS_on_scene']

# Add mission duration for pediatric patients
pediatric_df_model['mission_duration'] = (
    pd.to_datetime(pediatric_df_model['Übergabezeit'], format='%d.%m.%Y %H:%M:%S') - 
    pd.to_datetime(pediatric_df_model['Erstbefund'], format='%d.%m.%Y %H:%M:%S')
).dt.total_seconds() / 60

# Add the 3 required variables for pediatric analysis
pediatric_df_model['short_scene_time'] = (pediatric_df_model['mission_duration'] < 30).astype(int)

# Create medication dose variables (matching Table 1 approach)
pediatric_df_model['fentanyl_dose'] = 0
pediatric_df_model['ketamine_dose'] = 0
pediatric_df_model['esketamine_dose'] = 0
pediatric_df_model['morphine_dose'] = 0
pediatric_df_model['Alle Medikamente'] = pediatric_df_model['Alle Medikamente'].str.replace(',', ';')  # replace commas with semicolons for consistency
for i, row in pediatric_df_model.iterrows():
    if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
        continue
    for analgetic in row['Alle Medikamente'].split(';'):
        if analgetic.strip() == '':
            continue
        # remove mcg or mg from dose
        if '7IE' in analgetic:
                print(f"Skipping dose with 7IE: {analgetic}")
                continue

        analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
        if 'Fentanyl' in analgetic and '/h' not in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip()
            pediatric_df_model.at[i, 'fentanyl_dose'] += float(dose) 
        elif 'Fentanyl' in analgetic and '/h' in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
            dose = float(dose) * pediatric_df_model.at[i, 'mission_duration']  
            pediatric_df_model.at[i, 'fentanyl_dose'] += float(dose)
        elif 'Ketamin' in analgetic or 'Ketamine' in analgetic:
            dose = analgetic.split('Ketamin')[-1].strip()
            pediatric_df_model.at[i, 'ketamine_dose'] += float(dose)
        elif 'Esketamin' in analgetic:
            dose = analgetic.split('Esketamin')[-1].strip()
            pediatric_df_model.at[i, 'esketamine_dose'] += float(dose)
        elif 'Morphin' in analgetic or 'Morphine' in analgetic:
            dose = analgetic.split('Morphin')[-1].strip()
            pediatric_df_model.at[i, 'morphine_dose'] += float(dose)

# fentanyl given
pediatric_df_model['fentanyl_given'] = pediatric_df_model['fentanyl_dose'] > 0
pediatric_df_model['morphine_given'] = pediatric_df_model['morphine_dose'] > 0
pediatric_df_model['ketamine_given'] = pediatric_df_model['ketamine_dose'] > 0
pediatric_df_model['esketamine_given'] = pediatric_df_model['esketamine_dose'] > 0

# Create combined medication variables
pediatric_df_model['any_opiate_dose'] = pediatric_df_model['morphine_dose'] + pediatric_df_model['fentanyl_dose']
pediatric_df_model['any_opiate_given'] = (pediatric_df_model['morphine_dose'] > 0) | (pediatric_df_model['fentanyl_dose'] > 0)
pediatric_df_model['any_ketamine_given'] = (pediatric_df_model['ketamine_dose'] > 0) | (pediatric_df_model['esketamine_dose'] > 0)

# Variables for pediatric analysis (including the 3 required variables)
pediatric_variables = [
    'age',  # Age (continuous, but within pediatric range)
    'male_patient',  # Patient sex
    'male_physician',  # Physician sex (REQUIRED)
    'mission_duration',  # Short time at scene before transport (REQUIRED)
    'no_analgesic',  # Patients who receive no analgesic (REQUIRED)
    'primary_mission',  # Mission type
    'night_mission',  # Night vs day mission
    'trauma',  # Trauma vs non-trauma
    'winch_extraction',  # Winch extraction
    'vas_scene'  # VAS at scene
]

# Check available variables in pediatric dataset
available_pediatric_vars = [var for var in pediatric_variables if var in pediatric_df_model.columns]
missing_pediatric_vars = [var for var in pediatric_variables if var not in pediatric_df_model.columns]

print(f"\nAvailable variables for pediatric analysis: {len(available_pediatric_vars)}")
for var in available_pediatric_vars:
    print(f"- {var}")
    
if missing_pediatric_vars:
    print(f"\nMissing variables: {missing_pediatric_vars}")

# Create clean pediatric dataset
pediatric_df_clean = pediatric_df_model[['insufficient_pain_mgmt'] + available_pediatric_vars].dropna()

print(f"\nPediatric dataset after removing missing values: {len(pediatric_df_clean)} patients")
print(f"Insufficient pain management in pediatric patients: {pediatric_df_clean['insufficient_pain_mgmt'].sum()}/{len(pediatric_df_clean)} ({pediatric_df_clean['insufficient_pain_mgmt'].mean():.1%})")

In [ ]:
# Pediatric patients: Univariate and Multivariate Analysis

# Perform univariate analysis for pediatric patients
print("Performing univariate analysis for pediatric patients...")
pediatric_univariate_results = pd.DataFrame()

for var in available_pediatric_vars:
    try:
        result = univariate_logistic_regression(pediatric_df_clean, 'insufficient_pain_mgmt', [var])
        pediatric_univariate_results = pd.concat([pediatric_univariate_results, result], ignore_index=True)
    except Exception as e:
        print(f"Error in univariate analysis for {var}: {e}")

print(f"Completed univariate analysis for {len(pediatric_univariate_results)} variables")

# Perform multivariate analysis for pediatric patients
print("\nPerforming multivariate analysis for pediatric patients...")
try:
    pediatric_multivariate_results, pediatric_model_stats, pediatric_fitted_model = multivariate_logistic_regression(
        pediatric_df_clean, 
        'insufficient_pain_mgmt',
        available_pediatric_vars
    )
    print(f"Multivariate analysis completed with {len(pediatric_multivariate_results)} variables")
except Exception as e:
    print(f"Error in multivariate analysis: {e}")
    pediatric_multivariate_results = pd.DataFrame()

# Display pediatric results summary
print(f"\nPEDIATRIC ANALYSIS SUMMARY:")
print(f"Population: {len(pediatric_df_clean)} patients")
print(f"Variables analyzed: {len(available_pediatric_vars)}")
print(f"Univariate results: {len(pediatric_univariate_results)} variables")
print(f"Multivariate results: {len(pediatric_multivariate_results)} variables")

In [ ]:
# Create Table 2B: Pediatric Patients
if len(pediatric_multivariate_results) > 0:
    pediatric_table2 = create_table2(pediatric_univariate_results, pediatric_multivariate_results)

    print("Table 2B. Factors for Insufficient Pain Management - PEDIATRIC PATIENTS (<16 years)")
    print("=" * 80)
    print(f"{'Variable':<30} {'Univariate':<20} {'P-value':<10} {'Multivariate':<20} {'P-value':<10}")
    print(f"{'':30} {'OR (95% CI)':<20} {'':10} {'OR (95% CI)':<20} {'':10}")
    print("-" * 80)

    for _, row in pediatric_table2.iterrows():
        print(f"{row['Variable']:<30} {row['Univariate_OR_CI']:<20} {row['Univariate_P']:<10} {row['Multivariate_OR_CI']:<20} {row['Multivariate_P']:<10}")

    print("\nOR = Odds Ratio, CI = Confidence Interval")
    print(f"Model includes {len(pediatric_df_clean)} pediatric patients")
    print(f"Insufficient pain management: {pediatric_df_clean['insufficient_pain_mgmt'].sum()}/{len(pediatric_df_clean)} ({pediatric_df_clean['insufficient_pain_mgmt'].mean():.1%})")

    # Show significant variables for pediatric patients
    pediatric_significant_vars = pediatric_multivariate_results[pediatric_multivariate_results['P_value'] < 0.05]
    if len(pediatric_significant_vars) > 0:
        print(f"\nSignificant Variables in Pediatric Multivariate Model (p < 0.05):")
        for _, row in pediatric_significant_vars.iterrows():
            direction = "increased" if row['OR'] > 1 else "decreased"
            print(f"- {row['Variable']}: OR = {row['OR']:.2f}, {direction} odds of insufficient pain management")
    else:
        print("\nNo variables reached statistical significance (p < 0.05) in pediatric model")

    display(pediatric_table2)
else:
    print("Unable to create pediatric Table 2 due to multivariate analysis errors")
    print("This may be due to small sample size or limited variation in pediatric population")

In [ ]:
# save tables
# adult_table2.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/table2A_adult_results.csv', index=False)
# pediatric_table2.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/table2B_pediatric_results.csv', index=False)

In [ ]:
# Model Validation and Comparison

print("=" * 80)
print("MODEL VALIDATION AND POPULATION COMPARISON")
print("=" * 80)

# Define validation function
def validate_model(df, predictor_vars, outcome_var):
    """
    Validate the logistic regression model
    """
    from sklearn.metrics import classification_report, roc_auc_score
    
    # Recreate the model for validation
    X = df[predictor_vars]
    y = df[outcome_var]
    X_with_const = sm.add_constant(X)
    
    model = sm.Logit(y, X_with_const).fit(disp=0)
    y_pred_proba = model.predict(X_with_const)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Calculate metrics
    auc_score = roc_auc_score(y, y_pred_proba)
    class_report = classification_report(y, y_pred, output_dict=True)
    
    print(f"AUC: {auc_score:.3f}")
    print(f"Accuracy: {class_report['accuracy']:.3f}")
    print(f"Sensitivity: {class_report['1']['recall']:.3f}")
    print(f"Specificity: {class_report['0']['recall']:.3f}")
    
    return {
        'auc': auc_score,
        'accuracy': class_report['accuracy'],
        'sensitivity': class_report['1']['recall'],
        'specificity': class_report['0']['recall']
    }

# Adult model validation
if len(adult_multivariate_results) > 0:
    print("\nADULT MODEL VALIDATION:")
    adult_model_validation = validate_model(adult_df_clean, adult_multivariate_vars, 'insufficient_pain_mgmt')
    
# Pediatric model validation  
if len(pediatric_multivariate_results) > 0:
    print("\nPEDIATRIC MODEL VALIDATION:")
    pediatric_model_validation = validate_model(pediatric_df_clean, available_pediatric_vars, 'insufficient_pain_mgmt')

# Population comparison
print("\n" + "=" * 80)
print("POPULATION COMPARISON SUMMARY")
print("=" * 80)

print(f"\nSample Sizes:")
print(f"- Adult patients (≥16 years): {len(adult_df_clean)}")
print(f"- Pediatric patients (<16 years): {len(pediatric_df_clean)}")

print(f"\nInsufficient Pain Management Rates:")
if len(adult_df_clean) > 0:
    print(f"- Adults: {adult_df_clean['insufficient_pain_mgmt'].mean():.1%}")
if len(pediatric_df_clean) > 0:
    print(f"- Pediatric: {pediatric_df_clean['insufficient_pain_mgmt'].mean():.1%}")

# Compare significant variables between populations
if len(adult_multivariate_results) > 0 and len(pediatric_multivariate_results) > 0:
    adult_sig = set(adult_multivariate_results[adult_multivariate_results['P_value'] < 0.05]['Variable'])
    pediatric_sig = set(pediatric_multivariate_results[pediatric_multivariate_results['P_value'] < 0.05]['Variable'])
    
    print(f"\nSignificant Variables by Population:")
    print(f"- Adult-specific: {adult_sig - pediatric_sig}")
    print(f"- Pediatric-specific: {pediatric_sig - adult_sig}")
    print(f"- Common to both: {adult_sig & pediatric_sig}")

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE - SEPARATE ADULT AND PEDIATRIC TABLES GENERATED")
print("=" * 80)

## Summary Figures for Table 2

Visual representations of the multivariate analysis results to better understand factors associated with insufficient pain management in both adult and pediatric populations.

In [ ]:
adult_multivariate_results

In [ ]:
# Figure 1: Forest Plot - Odds Ratios Comparison (Adult vs Pediatric)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style for publication-quality figures
plt.style.use('default')
sns.set_palette("husl")

def create_forest_plot():
    """
    Create a forest plot comparing odds ratios between adult and pediatric populations
    """
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Prepare data for forest plot
    variables = []
    adult_ors = []
    adult_ci_lower = []
    adult_ci_upper = []
    adult_pvals = []
    
    pediatric_ors = []
    pediatric_ci_lower = []
    pediatric_ci_upper = []
    pediatric_pvals = []
    
    # Variable labels for better presentation
    var_labels = {
        'age': 'Age (years)',
        'male_patient': 'Male patient',
        'male_physician': 'Male physician',
        'short_scene_time': 'Short time at scene (<30 min)',
        'no_analgesic': 'No analgesic given',
        'primary_mission': 'Primary mission',
        'night_mission': 'Night mission',
        'trauma': 'Trauma',
        'winch_extraction': 'Winch extraction',
        'vas_scene': 'VAS at scene'
    }
    
    # Get common variables between adult and pediatric analyses
    if len(adult_multivariate_results) > 0 and len(pediatric_multivariate_results) > 0:
        adult_vars = set(adult_multivariate_results['Variable'])
        pediatric_vars = set(pediatric_multivariate_results['Variable'])
        common_vars = adult_vars.intersection(pediatric_vars)
        
        for var in sorted(common_vars):
            variables.append(var_labels.get(var, var))
            
            # Adult data
            adult_row = adult_multivariate_results[adult_multivariate_results['Variable'] == var].iloc[0]
            adult_ors.append(adult_row['OR'])
            adult_ci_lower.append(adult_row['CI_lower'])
            adult_ci_upper.append(adult_row['CI_upper'])
            adult_pvals.append(adult_row['P_value'])
            
            # Pediatric data
            pediatric_row = pediatric_multivariate_results[pediatric_multivariate_results['Variable'] == var].iloc[0]
            pediatric_ors.append(pediatric_row['OR'])
            pediatric_ci_lower.append(pediatric_row['CI_lower'])
            pediatric_ci_upper.append(pediatric_row['CI_upper'])
            pediatric_pvals.append(pediatric_row['P_value'])
    
    if variables:
        y_positions = np.arange(len(variables))
        colors1 = ['#FF6B6B', '#4ECDC4']
        # Plot adult ORs
        adult_colors = ['#FF6B6B' if p < 0.05 else '#FF6B6B' for p in adult_pvals]
        ax.errorbar(adult_ors, y_positions - 0.15, 
                   xerr=[np.array(adult_ors) - np.array(adult_ci_lower), 
                         np.array(adult_ci_upper) - np.array(adult_ors)],
                   fmt='o', color='#FF6B6B', capsize=5, capthick=2, 
                   label=f'Adult (n={len(adult_df_clean)})', markersize=8)
        
        # Plot pediatric ORs
        pediatric_colors = ['#4ECDC4' if p < 0.05 else 'light#4ECDC4blue' for p in pediatric_pvals]
        ax.errorbar(pediatric_ors, y_positions + 0.15, 
                   xerr=[np.array(pediatric_ors) - np.array(pediatric_ci_lower), 
                         np.array(pediatric_ci_upper) - np.array(pediatric_ors)],
                   fmt='s', color='#4ECDC4', capsize=5, capthick=2, 
                   label=f'Pediatric (n={len(pediatric_df_clean)})', markersize=8)
        
        # Formatting
        ax.axvline(x=1, color='black', linestyle='--', alpha=0.7)
        ax.set_xlabel('Odds Ratio (95% CI)', fontsize=12, fontweight='bold')
        ax.set_ylabel('Variables', fontsize=12, fontweight='bold')
        ax.set_yticks(y_positions)
        ax.set_yticklabels(variables)
        # ax.set_xscale('log')
        ax.grid(True, alpha=0.3)
        ax.legend(loc='upper right')
        ax.set_title('Figure 1: Multivariate Analysis - Factors Associated with Insufficient Pain Management\n(Odds Ratios with 95% Confidence Intervals)', 
                    fontsize=14, fontweight='bold', pad=20)
        
        # Add significance indicators
        # for i, (a_p, p_p) in enumerate(zip(adult_pvals, pediatric_pvals)):
        #     if a_p < 0.05:
        #         ax.text(max(adult_ors) * 1.5, i - 0.15, '*', fontsize=16, color='red', ha='center')
        #     if p_p < 0.05:
        #         ax.text(max(adult_ors) * 1.5, i + 0.15, '*', fontsize=16, color='blue', ha='center')
        
        plt.tight_layout()
        plt.show()
        
        print("Figure 1: Forest plot showing odds ratios for insufficient pain management")
        print("* indicates p < 0.05")
        print("Red = Adult patients, Blue = Pediatric patients")
    else:
        print("No common variables found between adult and pediatric analyses for forest plot")

# Create the forest plot
create_forest_plot()

In [ ]:
# Figure 4: Population Summary Dashboard
def create_population_summary_dashboard():
    """
    Create a comprehensive dashboard showing population characteristics and outcomes
    """
    fig = plt.figure(figsize=(16, 10))
    
    # Create a 2x3 grid for subplots
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # Subplot 1: Sample sizes
    ax1 = fig.add_subplot(gs[0, 0])
    populations = ['Adult\n(≥16 years)', 'Pediatric\n(<16 years)']
    sample_sizes = [len(adult_df_clean), len(pediatric_df_clean)]
    colors1 = ['#FF6B6B', '#4ECDC4']
    
    bars1 = ax1.bar(populations, sample_sizes, color=colors1, alpha=0.8)
    ax1.set_ylabel('Number of Patients', fontweight='bold')
    ax1.set_title('Sample Sizes', fontweight='bold', fontsize=11)
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Add values on bars
    for bar, size in zip(bars1, sample_sizes):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
                f'{size}', ha='center', va='bottom', fontweight='bold')
    
    # Subplot 2: Insufficient pain management rates
    ax2 = fig.add_subplot(gs[0, 1])
    adult_rate = adult_df_clean['insufficient_pain_mgmt'].mean() * 100 if len(adult_df_clean) > 0 else 0
    pediatric_rate = pediatric_df_clean['insufficient_pain_mgmt'].mean() * 100 if len(pediatric_df_clean) > 0 else 0
    
    rates = [adult_rate, pediatric_rate]
    bars2 = ax2.bar(populations, rates, color=colors1, alpha=0.8)
    ax2.set_ylabel('Insufficient Pain Management (%)', fontweight='bold')
    ax2.set_title('Outcome Rates', fontweight='bold', fontsize=11)
    ax2.set_ylim(0, 100)
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Add values on bars
    for bar, rate in zip(bars2, rates):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    # Subplot 3: Model performance comparison
    ax3 = fig.add_subplot(gs[0, 2])
    if len(adult_multivariate_results) > 0 and len(pediatric_multivariate_results) > 0:
        adult_perf = validate_model(adult_df_clean, adult_multivariate_vars, 'insufficient_pain_mgmt')
        pediatric_perf = validate_model(pediatric_df_clean, available_pediatric_vars, 'insufficient_pain_mgmt')
        
        metrics = ['AUC', 'Accuracy', 'Sensitivity', 'Specificity']
        adult_values = [adult_perf['auc'], adult_perf['accuracy'], adult_perf['sensitivity'], adult_perf['specificity']]
        pediatric_values = [pediatric_perf['auc'], pediatric_perf['accuracy'], pediatric_perf['sensitivity'], pediatric_perf['specificity']]
        
        x = np.arange(len(metrics))
        width = 0.35
        
        ax3.bar(x - width/2, adult_values, width, label='Adult', color=colors1[0], alpha=0.8)
        ax3.bar(x + width/2, pediatric_values, width, label='Pediatric', color=colors1[1], alpha=0.8)
        
        ax3.set_ylabel('Performance Score', fontweight='bold')
        ax3.set_title('Model Performance', fontweight='bold', fontsize=11)
        ax3.set_xticks(x)
        ax3.set_xticklabels(metrics, rotation=45)
        ax3.legend()
        ax3.grid(True, alpha=0.3, axis='y')
        ax3.set_ylim(0, 1)
    
    # Subplot 4: Significant variables count
    ax4 = fig.add_subplot(gs[1, :])
    
    # Count significant variables by population
    adult_sig_count = len(adult_multivariate_results[adult_multivariate_results['P_value'] < 0.05]) if len(adult_multivariate_results) > 0 else 0
    pediatric_sig_count = len(pediatric_multivariate_results[pediatric_multivariate_results['P_value'] < 0.05]) if len(pediatric_multivariate_results) > 0 else 0
    
    # Get all variables analyzed
    all_vars = set()
    if len(adult_multivariate_results) > 0:
        all_vars.update(adult_multivariate_results['Variable'])
    if len(pediatric_multivariate_results) > 0:
        all_vars.update(pediatric_multivariate_results['Variable'])
    
    var_labels = {
        'age': 'Age',
        'male_patient': 'Male patient',
        'male_physician': 'Male physician',
        'short_scene_time': 'Short time at scene',
        'no_analgesic': 'No analgesic given',
        'primary_mission': 'Primary mission', 
        'night_mission': 'Night mission',
        'trauma': 'Trauma',
        'winch_extraction': 'Winch extraction',
        'vas_scene': 'VAS at scene'
    }
    
    # Create significance matrix
    sig_matrix = []
    var_names = []
    
    for var in sorted(all_vars):
        var_names.append(var_labels.get(var, var))
        adult_sig = 1 if (len(adult_multivariate_results) > 0 and 
                         var in adult_multivariate_results['Variable'].values and
                         adult_multivariate_results[adult_multivariate_results['Variable'] == var]['P_value'].iloc[0] < 0.05) else 0
        pediatric_sig = 1 if (len(pediatric_multivariate_results) > 0 and 
                             var in pediatric_multivariate_results['Variable'].values and
                             pediatric_multivariate_results[pediatric_multivariate_results['Variable'] == var]['P_value'].iloc[0] < 0.05) else 0
        sig_matrix.append([adult_sig, pediatric_sig])
    
    if sig_matrix:
        sig_df = pd.DataFrame(sig_matrix, index=var_names, columns=['Adult', 'Pediatric'])
        
        im = ax4.imshow(sig_df.T, cmap='RdYlBu_r', aspect='auto', vmin=0, vmax=1)
        ax4.set_xticks(range(len(var_names)))
        ax4.set_xticklabels(var_names, rotation=45, ha='right')
        ax4.set_yticks(range(2))
        ax4.set_yticklabels(['Adult', 'Pediatric'])
        ax4.set_title('Significance Pattern (p < 0.05)', fontweight='bold', fontsize=11)
        
        # Add text annotations
        for i in range(2):
            for j in range(len(var_names)):
                text = '✓' if sig_df.iloc[j, i] == 1 else '✗'
                color = 'white' if sig_df.iloc[j, i] == 1 else 'black'
                ax4.text(j, i, text, ha='center', va='center', color=color, fontsize=14, fontweight='bold')
    
    # Subplot 5: Key statistics summary
    ax5 = fig.add_subplot(gs[2, :])
    ax5.axis('off')
    
    # Create summary text
    summary_text = f"""
    KEY FINDINGS SUMMARY:
    
    Adult Population (≥16 years):
    • Sample size: {len(adult_df_clean)} patients
    • Insufficient pain management: {adult_df_clean['insufficient_pain_mgmt'].sum()}/{len(adult_df_clean)} ({adult_df_clean['insufficient_pain_mgmt'].mean():.1%})
    • Significant variables: {adult_sig_count} factors (p < 0.05)
    
    Pediatric Population (<16 years):
    • Sample size: {len(pediatric_df_clean)} patients  
    • Insufficient pain management: {pediatric_df_clean['insufficient_pain_mgmt'].sum()}/{len(pediatric_df_clean)} ({pediatric_df_clean['insufficient_pain_mgmt'].mean():.1%})
    • Significant variables: {pediatric_sig_count} factors (p < 0.05)
    
    Clinical Significance:
    • {'Higher' if adult_rate > pediatric_rate else 'Lower'} insufficient pain management rate in adult vs pediatric patients
    • Age-specific risk factors may require different pain management approaches
    • Model performance suggests {'good' if (adult_perf.get('auc', 0) > 0.7 and pediatric_perf.get('auc', 0) > 0.7) else 'moderate'} discrimination ability
    """
    
    ax5.text(0.05, 0.95, summary_text, transform=ax5.transAxes, fontsize=10,
            verticalalignment='top', bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.1))
    
    plt.suptitle('Figure 4: Population Summary Dashboard - Insufficient Pain Management Analysis', 
                fontsize=16, fontweight='bold', y=0.95)
    plt.tight_layout()
    plt.show()
    
    print("Figure 4: Comprehensive population summary dashboard")
    print("✓ = Significant (p < 0.05), ✗ = Not significant")

# Create the population summary dashboard
create_population_summary_dashboard()